This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/pick.html). I recommend having both windows open, side-by-side!

In [2]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / 'drake'))

import numpy as np
from pydrake.all import (
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from ur5e_description.ur5e_station import MakeUR5eHardwareStation, GetPlantFromStation

In [3]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


# Our first end-effector "controller"

Let's use the pseudo-inverse of the Jacobian to drive the UR5e around. We compute $[J^G]^+$ to command a constant spatial velocity $V^G$. UR5e has 6 DOF so the Jacobian is 6x6 (no slicing needed unlike IIWA).

Try changing $V^G$ and see how it affects the robot motion!

In [ ]:
class PseudoInverseController(LeafSystem):
    """Computes joint velocities from desired end-effector velocity: qdot = J+ @ V_G"""
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()  # Separate context for Jacobian calc
        self._ur5e = plant.GetModelInstanceByName("ur5e")
        # TCP frame from NetFT probe (tool center point = end-effector)
        self._G = plant.GetBodyByName("tcp", plant.GetModelInstanceByName("netft_probe")).body_frame()
        self._W = plant.world_frame()

        # UR5e: 6 DOF (unlike IIWA's 7)
        self.DeclareVectorInputPort("ur5e.position", 6)
        self.DeclareVectorOutputPort("ur5e.velocity", 6, self.CalcOutput)

    def CalcOutput(self, context, output):
        q = self.get_input_port().Eval(context)
        self._plant.SetPositions(self._plant_context, self._ur5e, q)
        
        # J_G: 6x6 Jacobian - SQUARE! (no slicing needed unlike IIWA's 6x10)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot,
            self._G,
            [0, 0, 0],  # Point p_GP in TCP frame (origin)
            self._W,    # Frame A: world
            self._W,    # Frame E: express result in world
        )

        # V_G = [wx, wy, wz, vx, vy, vz] - desired spatial velocity of TCP
        V_G_desired = np.array([0, -0.1, 0, 0, -0.05, -0.1])
        
        # Core equation: qdot = J+ @ V_G (for 6x6, J+ ≈ J^-1 if not singular)
        v = np.linalg.pinv(J_G).dot(V_G_desired)
        output.SetFromVector(v)


def jacobian_controller_example(robot_ip=None):
    """
    Run pseudoinverse controller.
    - robot_ip=None → simulation
    - robot_ip="192.168.1.102" → real hardware (uses ur_rtde)
    """
    # === BUILD PHASE: Wire up systems ===
    builder = DiagramBuilder()

    # MakeUR5eHardwareStation: unified API for sim/hardware (unlike IIWA's separate paths)
    station = builder.AddSystem(MakeUR5eHardwareStation(
        robot_ip=robot_ip, meshcat=meshcat, control_mode="position"
    ))
    
    # GetPlantFromStation = station.GetSubsystemByName("plant") - convenience wrapper
    plant = GetPlantFromStation(station)

    controller = builder.AddSystem(PseudoInverseController(plant))
    integrator = builder.AddSystem(Integrator(6))  # ∫velocity dt → position

    # Wiring: controller(vel) → integrator → station(pos) → position_measured → controller
    builder.Connect(controller.get_output_port(), integrator.get_input_port())
    builder.Connect(integrator.get_output_port(), station.GetInputPort("ur5e.position"))
    builder.Connect(station.GetOutputPort("ur5e.position_measured"), controller.get_input_port())

    diagram = builder.Build()  # Finalize - no more changes allowed

    # === SIMULATION PHASE: Create context and initialize ===
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()  # Root context (holds all state)

    # Get initial position from plant (uses default from dmd.yaml: upright pose)
    # No FixValue needed! (unlike IIWA which needs torque + gripper FixValue)
    q0 = plant.GetPositions(plant.GetMyContextFromRoot(context))
    
    # Sync integrator state with current robot position (avoid discontinuity at t=0)
    integrator.set_integral_value(integrator.GetMyContextFromRoot(context), q0)

    # === RUN: Record and simulate ===
    meshcat.StartRecording(set_visualizations_while_recording=False)
    simulator.AdvanceTo(5.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()


# Run in simulation (for hardware: jacobian_controller_example(robot_ip="192.168.1.102"))
jacobian_controller_example()